In [30]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from os import path

In [31]:
data = pd.read_csv("cumulative.csv")

In [32]:
data.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [36]:
drop_columns = ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition','koi_tce_delivname']

In [37]:
df = data.drop(columns = drop_columns)

In [38]:
df = df.dropna(axis = "columns", how = "all")

In [39]:
df = df.dropna()

In [40]:
df["koi_pdisposition"].value_counts()

CANDIDATE         3996
FALSE POSITIVE    3807
Name: koi_pdisposition, dtype: int64

In [41]:
df.columns

Index(['koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff',
       'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1',
       'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra',
       'dec', 'koi_kepmag'],
      dtype='object')

In [42]:
df.head()

,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [43]:
# Assign X (data) and y (target)
X = df.drop("koi_pdisposition", axis=1)
y = df["koi_pdisposition"]
print(X.shape, y.shape)

(7803, 41) (7803,)


In [44]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## MinMax Scaler

In [45]:
#Fit on the training data
X_scaler = MinMaxScaler().fit(X_train)
X_scaler.get_params()

{'clip': False, 'copy': True, 'feature_range': (0, 1)}

In [47]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[0.999     , 0.        , 0.        , ..., 0.66362782, 0.84637293,
        0.81153254],
       [0.997     , 0.        , 0.        , ..., 0.40754724, 0.62296282,
        0.78872092],
       [0.        , 0.        , 0.        , ..., 0.36489169, 0.16733163,
        0.61473703],
       ...,
       [0.146     , 1.        , 0.        , ..., 0.68831833, 0.43179568,
        0.81117045],
       [0.943     , 0.        , 0.        , ..., 0.29930692, 0.75797076,
        0.7740563 ],
       [0.        , 0.        , 0.        , ..., 0.56064804, 0.70768092,
        0.78238436]])

In [48]:
#Create a Logistic Regression Model
classifier = LogisticRegression()
classifier

LogisticRegression()

In [49]:
y_train.value_counts()

CANDIDATE         2990
FALSE POSITIVE    2862
Name: koi_pdisposition, dtype: int64

In [50]:
y_test.value_counts()

CANDIDATE         1006
FALSE POSITIVE     945
Name: koi_pdisposition, dtype: int64

In [51]:
#Fit (train) or model using the training data
classifier.fit(X_train_scaled,y_train)

LogisticRegression()

In [52]:
#Validate the model using the test data
#classifier.score makes predictions
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.4890635680109364
Testing Data Score: 0.4843669912865197


## Grid Search

In [53]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'penalty': ["l1", "l2"]}
model = LogisticRegression(solver = "liblinear")
grid = GridSearchCV(model, param_grid, verbose=3)

In [54]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END ................................C=1, penalty=l1; total time=   0.0s
[CV 2/5] END ................................C=1, penalty=l1; total time=   0.0s
[CV 3/5] END ................................C=1, penalty=l1; total time=   0.0s
[CV 4/5] END ................................C=1, penalty=l1; total time=   0.0s
[CV 5/5] END ................................C=1, penalty=l1; total time=   0.0s
[CV 1/5] END ................................C=1, penalty=l2; total time=   0.0s
[CV 2/5] END ................................C=1, penalty=l2; total time=   0.0s
[CV 3/5] END ................................C=1, penalty=l2; total time=   0.0s
[CV 4/5] END ................................C=1, penalty=l2; total time=   0.0s
[CV 5/5] END ................................C=1, penalty=l2; total time=   0.0s
[CV 1/5] END ................................C=5, penalty=l1; total time=   0.0s
[CV 2/5] END ................................C=5,

GridSearchCV(estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': [1, 5, 10, 50], 'penalty': ['l1', 'l2']},
             verbose=3)

In [55]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 1, 'penalty': 'l1'}


In [56]:
# List the best score
print(grid.best_score_)

0.9991457370791273


In [57]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [58]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

Test Acc: 0.962
